**Figure 14 (Normalized Performance): 5-entry PSQ, 32 NBO, PRAC-1**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/QPRAC/4cores/500MInsts/MOP'

df = pd.DataFrame(columns=["workload"])
mitigation_list = ["Baseline", "QPRAC-NoOp","QPRAC", "QPRAC+Proactive"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        NBO = int(result_filename.split("_")[0])
        if NBO != 32:
            continue
        prac_level = int(result_filename.split("_")[1])
        if prac_level != 1:
            continue
        psq_size = int(result_filename.split("_")[2])
        if psq_size != 5:
            continue
        workload = "_".join(result_filename.split("_")[4:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        num_abo = 0
        num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" controller0_num_refresh_reqs" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        WS = ipc_0 + ipc_1 + ipc_2 + ipc_3
        ABO_per_tREFI = float(num_abo)/float(num_tREFI_period/2)

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'workload': [workload],
            'mitigation': [mitigation],
            'NBO': [NBO],
            'WS': [WS],
            'ABO_per_tREFI': [ABO_per_tREFI],
        })
        df = pd.concat([df, new_row], ignore_index=True)

df_ws = df.pivot(index=['workload', 'NBO'], columns=['mitigation'], values='WS').reset_index()
df_abo = df.pivot(index=['workload', 'NBO'], columns=['mitigation'], values='ABO_per_tREFI').reset_index()
df_abo.drop(columns=['Baseline'], inplace=True)

for mitigation in set(mitigation_list) - set(['Baseline']):
    df_ws[mitigation] = df_ws[mitigation] / df_ws['Baseline']
df_ws.drop(columns=['Baseline'], inplace=True)

# print(df_ws)

benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for NBO in df['NBO'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['NBO'] == NBO)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'NBO': NBO, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for NBO in df['NBO'].unique():
            Channel_interface_df = df[(df['NBO'] == NBO)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'NBO': NBO, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)


mitigation_list = ["QPRAC-NoOp","QPRAC", "QPRAC+Proactive"]
new_column_order = ['workload', 'NBO'] + mitigation_list

geomean_df = add_geomean_rows(df_ws)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
geomean_df = geomean_df[new_column_order]
print(geomean_df)
geomean_df.to_csv('../stats/4cores_QPRAC_32NBO_5PSQ_500MInsts.csv', index=False)

          workload   NBO  QPRAC-NoOp     QPRAC  QPRAC+Proactive
0        401.bzip2  32.0    0.902356  0.997772              1.0
1          403.gcc  32.0    1.000000  1.000000              1.0
2          429.mcf  32.0         NaN       NaN              NaN
3         433.milc  32.0    0.948515  0.997549              1.0
4       434.zeusmp  32.0    0.736759  0.963384              1.0
..             ...   ...         ...       ...              ...
59         TPC (4)  32.0    0.889154  0.997550              1.0
60      Hadoop (3)  32.0    0.990644  0.999279              1.0
61  MediaBench (3)  32.0    0.815713  0.986067              1.0
62        YCSB (6)  32.0    0.959188  0.997987              1.0
63        All (57)  32.0    0.874504  0.991259              1.0

[64 rows x 5 columns]


**Figure 15 (ABO per tREFI)**

In [4]:
# Function to calculate geometric mean
def calculate_arithmetic_mean(series):
    return series.mean()

# Function to calculate and add geometric means as new rows
def add_arithmetic_mean_rows(df):
    amean_rows = []  # List to collect new rows

    for NBO in df['NBO'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['NBO'] == NBO)]
            if not suite_df.empty:
                ameans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        ameans[mitigation] = calculate_arithmetic_mean(suite_df[mitigation])
                
                # Create a new row
                amean_row = {'NBO': NBO, 'workload': suite_name, **ameans}
                amean_rows.append(amean_row)  # Append to the list

    # Convert list of rows to DataFrame
    amean_df = pd.DataFrame(amean_rows)
    
    return pd.concat([df, amean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_amean_rows(df):
    amean_rows = []  # List to collect new rows
    
    for NBO in df['NBO'].unique():
            Channel_interface_df = df[(df['NBO'] == NBO)]
            amean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    amean_values[mitigation] = calculate_arithmetic_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            amean_row = {'NBO': NBO, 'workload': 'All (57)', **amean_values}
            amean_rows.append(amean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    amean_df = pd.DataFrame(amean_rows)
    
    return pd.concat([df, amean_df], ignore_index=True)

mitigation_list = ["QPRAC-NoOp","QPRAC", "QPRAC+Proactive"]
new_column_order = ['workload', 'NBO'] + mitigation_list

amean_df = add_arithmetic_mean_rows(df_abo)
amean_df = add_all_workloads_amean_rows(amean_df)
amean_df = amean_df[new_column_order]
print(amean_df)
amean_df.to_csv('../stats/4cores_QPRAC_32NBO_5PSQ_500MInsts_ABO_per_tREFI.csv', index=False)

          workload   NBO  QPRAC-NoOp     QPRAC  QPRAC+Proactive
0        401.bzip2  32.0    1.596443  0.056995              0.0
1          403.gcc  32.0    0.000000  0.000000              0.0
2          429.mcf  32.0         NaN       NaN              NaN
3         433.milc  32.0    0.480503  0.023194              0.0
4       434.zeusmp  32.0    2.262768  0.316365              0.0
..             ...   ...         ...       ...              ...
59         TPC (4)  32.0    1.152060  0.036749              0.0
60      Hadoop (3)  32.0    0.101241  0.003381              0.0
61  MediaBench (3)  32.0    1.549168  0.121235              0.0
62        YCSB (6)  32.0    0.487795  0.024992              0.0
63        All (57)  32.0    1.155248  0.080120              0.0

[64 rows x 5 columns]


**Figure 16 (Varying RFMs per ABO)**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/QPRAC/4cores/500MInsts/MOP'

df = pd.DataFrame(columns=["workload"])
df_baseline = pd.DataFrame(columns=["workload"])
mitigation_list = ["Baseline", "QPRAC", "QPRAC+Proactive"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        NBO = int(result_filename.split("_")[0])
        if NBO != 32:
            continue
        prac_level = int(result_filename.split("_")[1])
        psq_size = int(result_filename.split("_")[2])
        if psq_size != 5:
            continue
        workload = "_".join(result_filename.split("_")[4:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" controller0_num_refresh_reqs" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        WS = ipc_0 + ipc_1 + ipc_2 + ipc_3
        ABO_per_tREFI = float(num_abo)/float(num_tREFI_period/2)

        result_file.close()
        # Create a new DataFrame for the new row

        if mitigation in ['Baseline']:
            baseline_new_row = pd.DataFrame({
                'workload': [workload],
                'mitigation': [mitigation],
                'NBO': [NBO],
                'WS': [WS]
            })
            df_baseline = pd.concat([df_baseline, baseline_new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame({
                'workload': [workload],
                'mitigation': [mitigation],
                'PRAC_level': [prac_level],
                'NBO': [NBO],
                'WS': [WS],
                'ABO_per_tREFI': [ABO_per_tREFI],
            })
            df = pd.concat([df, new_row], ignore_index=True)

df_baseline = df_baseline.pivot(index=['workload'], columns=['mitigation'], values='WS').reset_index()
df_ws = df.pivot(index=['workload', 'PRAC_level'], columns=['mitigation'], values='WS').reset_index()

df_ws = df_ws.merge(df_baseline, on='workload', how='left')
for mitigation in set(mitigation_list) - set(['Baseline']):
    df_ws[mitigation] = df_ws[mitigation] / df_ws['Baseline']
df_ws.drop(columns=['Baseline'], inplace=True)

# print(df_ws)

benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for NBO in df['PRAC_level'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['PRAC_level'] == NBO)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'PRAC_level': NBO, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for NBO in df['PRAC_level'].unique():
            Channel_interface_df = df[(df['PRAC_level'] == NBO)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'PRAC_level': NBO, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)


mitigation_list = ["QPRAC", "QPRAC+Proactive"]
new_column_order = ['workload', 'PRAC_level'] + mitigation_list

geomean_df = add_geomean_rows(df_ws)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
geomean_df = geomean_df[new_column_order]
print(geomean_df)
geomean_df.to_csv('../stats/4cores_QPRAC_32NBO_5PSQ_500MInsts_varying_RFMs_per_ABO_performance.csv', index=False)

           workload  PRAC_level     QPRAC  QPRAC+Proactive
0         401.bzip2         1.0  0.997772              1.0
1         401.bzip2         2.0  0.997876              1.0
2         401.bzip2         4.0  0.995595              1.0
3           403.gcc         1.0  1.000000              1.0
4           403.gcc         2.0  1.000000              1.0
..              ...         ...       ...              ...
184  MediaBench (3)         4.0  0.977043              1.0
185        YCSB (6)         4.0  0.995813              1.0
186        All (57)         1.0  0.991109              1.0
187        All (57)         2.0  0.990693              1.0
188        All (57)         4.0  0.988691              1.0

[189 rows x 4 columns]


**Figure 17 (Varying PSQ sizes with varying Targeted Refresh ratio)**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/QPRAC/4cores/500MInsts/MOP'

df = pd.DataFrame(columns=["workload"])
df_baseline = pd.DataFrame(columns=["workload"])
mitigation_list = ["Baseline", "QPRAC", "QPRAC+1Proactive_per_4tREFI", "QPRAC+1Proactive_per_2tREFI", "QPRAC+1Proactive_per_1tREFI"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        NBO = int(result_filename.split("_")[0])
        if NBO != 32:
            continue
        prac_level = int(result_filename.split("_")[1])
        if prac_level != 1:
            continue
        psq_size = int(result_filename.split("_")[2])
        workload = "_".join(result_filename.split("_")[4:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" controller0_num_refresh_reqs" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        WS = ipc_0 + ipc_1 + ipc_2 + ipc_3
        ABO_per_tREFI = float(num_abo)/float(num_tREFI_period/2)

        result_file.close()
        # Create a new DataFrame for the new row

        if mitigation in ['Baseline']:
            baseline_new_row = pd.DataFrame({
                'workload': [workload],
                'mitigation': [mitigation],
                'NBO': [NBO],
                'WS': [WS]
            })
            df_baseline = pd.concat([df_baseline, baseline_new_row], ignore_index=True)
        else:
            new_row = pd.DataFrame({
                'workload': [workload],
                'mitigation': [mitigation],
                'PSQ_size': [psq_size],
                'NBO': [NBO],
                'WS': [WS],
                'ABO_per_tREFI': [ABO_per_tREFI],
            })
            df = pd.concat([df, new_row], ignore_index=True)

df_baseline = df_baseline.pivot(index=['workload'], columns=['mitigation'], values='WS').reset_index()
df_ws = df.pivot(index=['workload', 'PSQ_size'], columns=['mitigation'], values='WS').reset_index()

df_ws = df_ws.merge(df_baseline, on='workload', how='left')
for mitigation in set(mitigation_list) - set(['Baseline']):
    df_ws[mitigation] = df_ws[mitigation] / df_ws['Baseline']
df_ws.drop(columns=['Baseline'], inplace=True)

# print(df_ws)

benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for NBO in df['PSQ_size'].unique():
        for suite_name, workloads in benchmark_suites.items():
            suite_df = df[(df['workload'].isin(workloads)) & (df['PSQ_size'] == NBO)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    if mitigation in suite_df.columns:  # Ensure the column exists
                        geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'PSQ_size': NBO, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for NBO in df['PSQ_size'].unique():
            Channel_interface_df = df[(df['PSQ_size'] == NBO)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'PSQ_size': NBO, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)


mitigation_list = ["QPRAC", "QPRAC+1Proactive_per_4tREFI", "QPRAC+1Proactive_per_2tREFI", "QPRAC+1Proactive_per_1tREFI"]
new_column_order = ['workload', 'PSQ_size'] + mitigation_list

geomean_df = add_geomean_rows(df_ws)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
geomean_df = geomean_df[new_column_order]
print(geomean_df)
geomean_df.to_csv('../stats/4cores_QPRAC_32NBO_500MInsts_varying_PSQ_sizes_performance.csv', index=False)

      workload  PSQ_size     QPRAC  QPRAC+1Proactive_per_4tREFI  \
0    401.bzip2       1.0  0.969013                     1.000000   
1    401.bzip2       2.0  0.996357                     1.000000   
2    401.bzip2       3.0  0.991076                     1.000000   
3    401.bzip2       4.0  0.998469                     1.000000   
4    401.bzip2       5.0  0.997772                     1.000000   
..         ...       ...       ...                          ...   
310   All (57)       1.0  0.937153                     0.972831   
311   All (57)       2.0  0.986540                     0.994966   
312   All (57)       3.0  0.989529                     0.996934   
313   All (57)       4.0  0.990682                     0.997509   
314   All (57)       5.0  0.991109                     0.998440   

     QPRAC+1Proactive_per_2tREFI  QPRAC+1Proactive_per_1tREFI  
0                       1.000000                          1.0  
1                       1.000000                          1.0  
2  

**352 ROB OPT MC**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
baseline_4cores_out_path = '../../prac_study/TimingOverhead/SC/OPTMC/352rob'

df = pd.DataFrame(columns=["mitigation", "workload", "WS"])
mitigation_list = ["Baseline", "PRAC_WO_Mitigation", 
                   "Baseline-ClosedCap4", "PRAC_WO_Mitigation-ClosedCap4"]
for mitigation in mitigation_list:
    result_path = baseline_4cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])

        workload = "_".join(result_filename.split("_")[2:])

        ipc_0 = 0
        cycle_0 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_abo=0
        num_tREFI_period=0
        num_tREFW_period=0
        num_rd_reqs=0
        num_wr_reqs=0
        for line in result_file.readlines():
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" num_refresh_command_0" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            if (" num_refresh_window_0" in line):
                num_tREFW_period = int(line.split(" ")[-1])            
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])

        if (cycle_0 == 0):
            continue
        if (cycle_0 == 0 ):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'WS': [ipc_0],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio]
        })
        df = pd.concat([df, new_row], ignore_index=True)

df[['workload', 'Channel', 'interface', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./../stats/SC_500M_OPTMC_workload_characteristics.csv', index=False)
df_baseline = df.pivot(index=['workload', 'Channel', 'interface'], columns=['mitigation'], values='WS').reset_index()
df_baseline[['workload', 'Channel','interface', 'Baseline','Baseline-ClosedCap4']].to_csv('./../stats/SC_500M_OPTMC_Baseline.csv', index=False)

# List of PRAC_WO_Mitigation columns and their corresponding Baseline columns
prac_columns = [
    'PRAC_WO_Mitigation',
    # 'PRAC_WO_Mitigation-ClosedCap1',
    'PRAC_WO_Mitigation-ClosedCap4'
]

baseline_columns = [
    'Baseline',
    # 'Baseline-ClosedCap1',
    'Baseline-ClosedCap4'
]

# Loop through each pair of PRAC and Baseline columns to create normalized columns
for prac_col, baseline_col in zip(prac_columns, baseline_columns):
    normalized_col_name = f'Normalized_{prac_col}'  # Name of the new normalized column
    df_baseline[normalized_col_name] = df_baseline[prac_col] / df_baseline[baseline_col]  # Calculate normalized values

# print(df_baseline)

# workload_interests = ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '482.sphinx3', '483.xalancbmk', 
#                       '500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz',
#                       'tpcc64', 'tpch17', 'tpch2', 'tpch6',
#                       'grep_map0', 'wc_8443', 'wc_map0',
#                       'h264_encode', 'jp2_decode', 'jp2_encode',
#                       'ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver',
#                       'random_10.trace', 'stream_10.trace'
#                       ]

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

#
mitigation_list = ["Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap1","Normalized_PRAC_WO_Mitigation-ClosedCap4"]

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, workloads in benchmark_suites.items():
                suite_df = df[(df['workload'].isin(workloads)) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)
# Call function to calculate and merge geometric means
geomean_df = add_geomean_rows(df_baseline)
print(geomean_df)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
geomean_df[['workload', 'Channel', 'interface', "Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap4"]].to_csv('./../stats/SC_500M_OPTMC_result.csv', index=False)

           workload  Channel  interface  Baseline  Baseline-ClosedCap4  \
0         401.bzip2      1.0     3200.0  3.291958             3.294998   
1         401.bzip2      1.0     6400.0  3.381844             3.382914   
2         401.bzip2      1.0     8800.0  3.401019             3.403233   
3         401.bzip2      1.0    12800.0  3.411890             3.412616   
4         401.bzip2      1.0    17600.0  3.426491             3.427592   
..              ...      ...        ...       ...                  ...   
562   SPEC2K17 (18)     16.0     8800.0       NaN                  NaN   
563         TPC (4)     16.0     8800.0       NaN                  NaN   
564      Hadoop (3)     16.0     8800.0       NaN                  NaN   
565  MediaBench (3)     16.0     8800.0       NaN                  NaN   
566        YCSB (6)     16.0     8800.0       NaN                  NaN   

     PRAC_WO_Mitigation  PRAC_WO_Mitigation-ClosedCap4  \
0              3.218930                       3.22195

ZeroDivisionError: division by zero

**4-Core, MOP, 500M, Open Page, Varying Channel**

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/TimingOverhead/4cores_results/OPTMC'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline","PRAC_WO_Mitigation"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])
        if interface != 8800:
            continue
        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# print(df)
df[['workload', 'interface', 'Channel', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./../stats/4core_OPTMC_heterogeneous_workload_characteristics.csv', index=False)

df_sc_ipc = pd.read_csv('../stats/SC_500M_OPTMC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['interface'] == 8800)]
# df_sc_ipc = df_sc_ipc.drop(columns=['interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl0'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl1'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl2'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline']], left_on=['Channel', 'wl3'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_closed_cap1_ws = df[['mitigation', 'workload', 'Channel', 'WS']]

df_closed_cap1_ws_pivot = df_closed_cap1_ws.pivot(index=['workload', 'Channel'], columns=['mitigation'], values='WS').reset_index()
for mitigation in set(mitigation_list) - set(['Baseline']):
     df_closed_cap1_ws_pivot[mitigation] = df_closed_cap1_ws_pivot[mitigation] / df_closed_cap1_ws_pivot['Baseline']
df_closed_cap1_ws_pivot.drop(columns=['Baseline'], inplace=True)
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_closed_cap1_ws_pivot['Mix_index'] = df_closed_cap1_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_closed_cap1_ws_pivot['Benchmark_Types'] = df_closed_cap1_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def geom_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for suite_name, mix_indices in benchmark_types.items():
            # Create a list of corresponding workload names (e.g., MIX0, MIX1)
            workloads = [f'MIX{i}' for i in mix_indices]
            suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'Channel': Channel, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        Channel_df = df[df['Channel'] == Channel]
        geomean_values = {}
        
        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'Channel': Channel, 'workload': 'All (90)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["PRAC_WO_Mitigation"]

geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', 'PRAC_WO_Mitigation', "Benchmark_Types"]].to_csv('../stats/4cores_OPTMC_DDR58800_500M_Heterogeneous_OpenPage_varying_channels.csv', index=False)


     workload  Channel  PRAC_WO_Mitigation  Mix_index Benchmark_Types
0        MIX0      1.0            0.857117        0.0       HHHH (15)
1        MIX0      2.0            0.842962        0.0       HHHH (15)
2        MIX0      4.0            0.846322        0.0       HHHH (15)
3        MIX0      8.0            0.868094        0.0       HHHH (15)
4        MIX0     16.0            0.875079        0.0       HHHH (15)
..        ...      ...                 ...        ...             ...
479  All (90)      1.0            0.901812        NaN             NaN
480  All (90)      2.0            0.905765        NaN             NaN
481  All (90)      4.0            0.919181        NaN             NaN
482  All (90)      8.0            0.931679        NaN             NaN
483  All (90)     16.0            0.941217        NaN             NaN

[484 rows x 5 columns]


**4-Core, MOP, 500M, Open Page, Varying Interface**

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/TimingOverhead/4cores_results/OPTMC'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline", "PRAC_WO_Mitigation"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])
        if Channel != 1:
            continue
        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# print(df)
df[['workload', 'interface', 'Channel', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./../stats/4core_OPTMC_heterogeneous_workload_characteristics.csv', index=False)

df_sc_ipc = pd.read_csv('../stats/SC_500M_OPTMC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel'] == 1)]
# df_sc_ipc = df_sc_ipc.drop(columns=['interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

# Calculate normalized IPC for 'Baseline' and 'PRAC_WO_Mitigation' using ipc_wl
df['normalized_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalized_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalized_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalized_ipc3'] = df['ipc3'] / df['ipc_wl3']

# Calculate WS for each set
df['WS'] = df[['normalized_ipc0', 'normalized_ipc1', 'normalized_ipc2', 'normalized_ipc3']].sum(axis=1)


df_open_varying_interface_ws = df[['mitigation', 'workload', 'Channel', 'interface', 'WS']]

# Pivot WS columns
df_open_varying_interface_ws_pivot = df_open_varying_interface_ws.pivot(index=['workload', 'Channel', 'interface'], columns='mitigation', values='WS').reset_index()

# Normalize WS by 'Baseline'
for mitigation in set(mitigation_list) - set(['Baseline']):
    df_open_varying_interface_ws_pivot[mitigation] = df_open_varying_interface_ws_pivot[mitigation] / df_open_varying_interface_ws_pivot['Baseline']

# Drop Baseline columns
df_open_varying_interface_ws_pivot.drop(columns=['Baseline'], inplace=True)

# Merge results back if needed
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_open_varying_interface_ws_pivot['Mix_index'] = df_open_varying_interface_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_open_varying_interface_ws_pivot['Benchmark_Types'] = df_open_varying_interface_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def calculate_geometric_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, mix_indices in benchmark_types.items():
                # Create a list of corresponding workload names (e.g., MIX0, MIX1)
                workloads = [f'MIX{i}' for i in mix_indices]
                suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}
            
            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["PRAC_WO_Mitigation"]

geomean_df = add_geomean_rows(df_open_varying_interface_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', 'interface', 'PRAC_WO_Mitigation', "Benchmark_Types"]].to_csv('../stats/4cores_OPTMC_OpenPage_500M_Heterogeneous_varying_interface.csv', index=False)


     workload  Channel  interface  PRAC_WO_Mitigation  Mix_index  \
0        MIX0      1.0     3200.0            0.973599        0.0   
1        MIX0      1.0     6400.0            0.886445        0.0   
2        MIX0      1.0     8800.0            0.858497        0.0   
3        MIX0      1.0    12800.0            0.848931        0.0   
4        MIX0      1.0    17600.0            0.844092        0.0   
..        ...      ...        ...                 ...        ...   
475  All (57)      1.0     3200.0            0.964069        NaN   
476  All (57)      1.0     6400.0            0.922787        NaN   
477  All (57)      1.0     8800.0            0.908726        NaN   
478  All (57)      1.0    12800.0            0.902723        NaN   
479  All (57)      1.0    17600.0            0.901645        NaN   

    Benchmark_Types  
0         HHHH (15)  
1         HHHH (15)  
2         HHHH (15)  
3         HHHH (15)  
4         HHHH (15)  
..              ...  
475             NaN  
476    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../../prac_study/TimingOverhead/4cores_results/OPTMC'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline-ClosedCap4", "PRAC_WO_Mitigation-ClosedCap4"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])
        if Channel != 1:
            continue
        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# print(df)
df[['workload', 'interface', 'Channel', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./../stats/4core_OPTMC_heterogeneous_workload_characteristics.csv', index=False)

df_sc_ipc = pd.read_csv('../stats/SC_500M_OPTMC_Baseline.csv')
# Choose only interested baseline
df_sc_ipc = df_sc_ipc[['workload', 'Channel', 'interface', 'Baseline-ClosedCap4']]
df_sc_ipc = df_sc_ipc[(df_sc_ipc['Channel'] == 1)]
# df_sc_ipc = df_sc_ipc.drop(columns=['interface'])
df_sc_ipc = df_sc_ipc.rename(columns={'workload': 'workload_sc'})
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline-ClosedCap4']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap4': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline-ClosedCap4']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap4': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline-ClosedCap4']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap4': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'interface', 'workload_sc', 'Baseline-ClosedCap4']], left_on=['Channel', 'interface', 'wl0'], right_on=['Channel', 'interface', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap4': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

# Calculate normalized IPC for 'Baseline' and 'PRAC_WO_Mitigation' using ipc_wl
df['normalized_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalized_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalized_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalized_ipc3'] = df['ipc3'] / df['ipc_wl3']

# Calculate WS for each set
df['WS'] = df[['normalized_ipc0', 'normalized_ipc1', 'normalized_ipc2', 'normalized_ipc3']].sum(axis=1)


df_open_varying_interface_ws = df[['mitigation', 'workload', 'Channel', 'interface', 'WS']]

# Pivot WS columns
df_open_varying_interface_ws_pivot = df_open_varying_interface_ws.pivot(index=['workload', 'Channel', 'interface'], columns='mitigation', values='WS').reset_index()

# Normalize WS by 'Baseline'
for mitigation in set(mitigation_list) - set(['Baseline-ClosedCap4']):
    df_open_varying_interface_ws_pivot[mitigation] = df_open_varying_interface_ws_pivot[mitigation] / df_open_varying_interface_ws_pivot['Baseline-ClosedCap4']

# Drop Baseline columns
df_open_varying_interface_ws_pivot.drop(columns=['Baseline-ClosedCap4'], inplace=True)

# Merge results back if needed
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_open_varying_interface_ws_pivot['Mix_index'] = df_open_varying_interface_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_open_varying_interface_ws_pivot['Benchmark_Types'] = df_open_varying_interface_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def calculate_geometric_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, mix_indices in benchmark_types.items():
                # Create a list of corresponding workload names (e.g., MIX0, MIX1)
                workloads = [f'MIX{i}' for i in mix_indices]
                suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}
            
            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["PRAC_WO_Mitigation-ClosedCap4"]

geomean_df = add_geomean_rows(df_open_varying_interface_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', 'interface', 'PRAC_WO_Mitigation-ClosedCap4', "Benchmark_Types"]].to_csv('../stats/4cores_OPTMC_ClosedCap4_500M_Heterogeneous_varying_interface.csv', index=False)


**128 ROB NonOPT MC**

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
baseline_4cores_out_path = '../TimingOverhead/SC'

df = pd.DataFrame(columns=["mitigation", "workload", "WS"])
mitigation_list = ["Baseline", "PRAC_WO_Mitigation", 
                   "Baseline-ClosedCap4", "PRAC_WO_Mitigation-ClosedCap4",
                   "Baseline-ClosedCap1", "PRAC_WO_Mitigation-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = baseline_4cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])

        workload = "_".join(result_filename.split("_")[2:])

        ipc_0 = 0
        cycle_0 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_abo=0
        num_tREFI_period=0
        num_tREFW_period=0
        for line in result_file.readlines():
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" num_refresh_command_0" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            if (" num_refresh_window_0" in line):
                num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0):
            continue
        if (cycle_0 == 0 ):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'WS': [ipc_0]
        })
        df = pd.concat([df, new_row], ignore_index=True)

df_baseline = df.pivot(index=['workload', 'Channel', 'interface'], columns=['mitigation'], values='WS').reset_index()

# List of PRAC_WO_Mitigation columns and their corresponding Baseline columns
prac_columns = [
    'PRAC_WO_Mitigation',
    'PRAC_WO_Mitigation-ClosedCap1',
    'PRAC_WO_Mitigation-ClosedCap4'
]

baseline_columns = [
    'Baseline',
    'Baseline-ClosedCap1',
    'Baseline-ClosedCap4'
]

# Loop through each pair of PRAC and Baseline columns to create normalized columns
for prac_col, baseline_col in zip(prac_columns, baseline_columns):
    normalized_col_name = f'Normalized_{prac_col}'  # Name of the new normalized column
    df_baseline[normalized_col_name] = df_baseline[prac_col] / df_baseline[baseline_col]  # Calculate normalized values

# print(df_baseline)

# workload_interests = ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '482.sphinx3', '483.xalancbmk', 
#                       '500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz',
#                       'tpcc64', 'tpch17', 'tpch2', 'tpch6',
#                       'grep_map0', 'wc_8443', 'wc_map0',
#                       'h264_encode', 'jp2_decode', 'jp2_encode',
#                       'ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver',
#                       'random_10.trace', 'stream_10.trace'
#                       ]

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (22)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 22
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

#
mitigation_list = ["Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap1","Normalized_PRAC_WO_Mitigation-ClosedCap4"]

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, workloads in benchmark_suites.items():
                suite_df = df[(df['workload'].isin(workloads)) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (56)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)
# Call function to calculate and merge geometric means
geomean_df = add_geomean_rows(df_baseline)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', 'interface', "Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap1","Normalized_PRAC_WO_Mitigation-ClosedCap4"]].to_csv('./SC_500M_result.csv', index=False)

           workload  Channel  interface  Baseline  Baseline-ClosedCap1  \
0         401.bzip2      1.0     3200.0  2.815950             2.618095   
1         401.bzip2      1.0     6400.0  2.880786             2.717947   
2           403.gcc      1.0     3200.0  3.680948             3.688203   
3           403.gcc      1.0     6400.0  3.705629             3.714663   
4           429.mcf      1.0     3200.0  0.385007             0.312998   
..              ...      ...        ...       ...                  ...   
123      Hadoop (3)      1.0     6400.0       NaN                  NaN   
124  MediaBench (3)      1.0     6400.0       NaN                  NaN   
125        YCSB (6)      1.0     6400.0       NaN                  NaN   
126        All (56)      1.0     3200.0       NaN                  NaN   
127        All (56)      1.0     6400.0       NaN                  NaN   

     Baseline-ClosedCap4  PRAC_WO_Mitigation  PRAC_WO_Mitigation-ClosedCap1  \
0               2.812995        

**352 ROB NonOPT MC**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
baseline_4cores_out_path = '../../prac_study/TimingOverhead/SC/352rob'

df = pd.DataFrame(columns=["mitigation", "workload", "WS"])
mitigation_list = ["Baseline", "PRAC_WO_Mitigation", 
                   "Baseline-ClosedCap4", "PRAC_WO_Mitigation-ClosedCap4"]
for mitigation in mitigation_list:
    result_path = baseline_4cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])
        interface = int(result_filename.split("_")[1])

        workload = "_".join(result_filename.split("_")[2:])

        ipc_0 = 0
        cycle_0 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_abo=0
        num_tREFI_period=0
        num_tREFW_period=0
        num_rd_reqs=0
        num_wr_reqs=0
        for line in result_file.readlines():
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" prac_num_recovery" in line):
                num_abo = int(line.split(" ")[-1])
            if (" num_refresh_command_0" in line):
                num_tREFI_period = int(line.split(" ")[-1])
            if (" num_refresh_window_0" in line):
                num_tREFW_period = int(line.split(" ")[-1])            
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])

        if (cycle_0 == 0):
            continue
        if (cycle_0 == 0 ):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))

        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'interface': [interface],
            'WS': [ipc_0],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio]
        })
        df = pd.concat([df, new_row], ignore_index=True)

df[['workload', 'Channel', 'interface', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./stats/SC_workload_characteristics.csv', index=False)
df_baseline = df.pivot(index=['workload', 'Channel', 'interface'], columns=['mitigation'], values='WS').reset_index()
df_baseline[['workload', 'Channel','interface', 'Baseline','Baseline-ClosedCap4']].to_csv('./stats/SC_500M_Baseline.csv', index=False)

# List of PRAC_WO_Mitigation columns and their corresponding Baseline columns
prac_columns = [
    'PRAC_WO_Mitigation',
    # 'PRAC_WO_Mitigation-ClosedCap1',
    'PRAC_WO_Mitigation-ClosedCap4'
]

baseline_columns = [
    'Baseline',
    # 'Baseline-ClosedCap1',
    'Baseline-ClosedCap4'
]

# Loop through each pair of PRAC and Baseline columns to create normalized columns
for prac_col, baseline_col in zip(prac_columns, baseline_columns):
    normalized_col_name = f'Normalized_{prac_col}'  # Name of the new normalized column
    df_baseline[normalized_col_name] = df_baseline[prac_col] / df_baseline[baseline_col]  # Calculate normalized values

# print(df_baseline)

# workload_interests = ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '482.sphinx3', '483.xalancbmk', 
#                       '500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz',
#                       'tpcc64', 'tpch17', 'tpch2', 'tpch6',
#                       'grep_map0', 'wc_8443', 'wc_map0',
#                       'h264_encode', 'jp2_decode', 'jp2_encode',
#                       'ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver',
#                       'random_10.trace', 'stream_10.trace'
#                       ]

# Define benchmark suites and their corresponding workloads
benchmark_suites = {
    'SPEC2K6 (23)': ['401.bzip2', '403.gcc', '429.mcf', '433.milc', '434.zeusmp', '435.gromacs', '436.cactusADM', '437.leslie3d', '444.namd', '445.gobmk', '447.dealII', '450.soplex', '456.hmmer', '458.sjeng', '459.GemsFDTD', '462.libquantum', '464.h264ref', '470.lbm', '471.omnetpp', '473.astar', '481.wrf', '482.sphinx3', '483.xalancbmk'], # SPEC2K6: 23
    'SPEC2K17 (18)': ['500.perlbench', '502.gcc', '505.mcf', '507.cactuBSSN', '508.namd', '510.parest', '511.povray', '519.lbm', '520.omnetpp', '523.xalancbmk', '525.x264', '526.blender', '531.deepsjeng', '538.imagick', '541.leela', '544.nab', '549.fotonik3d', '557.xz'], # SPEC2K17: 18
    'TPC (4)': ['tpcc64', 'tpch17', 'tpch2', 'tpch6'], #tpc: 4
    # TODO: Enable Hadoop and LonestartGPU after fixing the performance shooting problem + h264_decode
    'Hadoop (3)': ['grep_map0', 'wc_8443', 'wc_map0'], #Hadoop: 3
    'MediaBench (3)': ['h264_encode', 'jp2_decode', 'jp2_encode'], #mediabench: 3
    'YCSB (6)': ['ycsb_abgsave', 'ycsb_aserver', 'ycsb_bserver', 'ycsb_cserver', 'ycsb_dserver', 'ycsb_eserver'] #ycsb:6
}

#
mitigation_list = ["Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap1","Normalized_PRAC_WO_Mitigation-ClosedCap4"]

# Function to calculate geometric mean
def calculate_geometric_mean(series):
    return np.prod(series) ** (1 / len(series))

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            for suite_name, workloads in benchmark_suites.items():
                suite_df = df[(df['workload'].isin(workloads)) & (df['Channel'] == Channel) & (df['interface'] == interface)]
                if not suite_df.empty:
                    geomeans = {}
                    
                    # Dynamically calculate geometric means for each mitigation
                    for mitigation in mitigation_list:
                        if mitigation in suite_df.columns:  # Ensure the column exists
                            geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                    
                    # Create a new row
                    geomean_row = {'Channel': Channel, 'interface': interface, 'workload': suite_name, **geomeans}
                    geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

# Function to add combined geometric means for all workloads in each channel and interface
def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        for interface in df['interface'].unique():  # Loop through unique interfaces
            Channel_interface_df = df[(df['Channel'] == Channel) & (df['interface'] == interface)]
            geomean_values = {}

            # Calculate geometric means for each mitigation in the list
            for mitigation in mitigation_list:
                if mitigation in Channel_interface_df.columns:  # Ensure the column exists
                    geomean_values[mitigation] = calculate_geometric_mean(Channel_interface_df[mitigation])

            # Create a new row for the combined results
            geomean_row = {'Channel': Channel, 'interface': interface, 'workload': 'All (57)', **geomean_values}
            geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)
# Call function to calculate and merge geometric means
geomean_df = add_geomean_rows(df_baseline)
# geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df[['workload', 'Channel', 'interface', "Normalized_PRAC_WO_Mitigation","Normalized_PRAC_WO_Mitigation-ClosedCap4"]].to_csv('./SC_500M_352robs_result.csv', index=False)

           workload  Channel  interface  Baseline  Baseline-ClosedCap4  \
0         401.bzip2      1.0     3200.0  3.283145             3.278445   
1         401.bzip2      1.0     6400.0  3.369152             3.364250   
2         401.bzip2      1.0     8800.0  3.392935             3.387190   
3         401.bzip2      1.0    12800.0  3.403206             3.397320   
4         401.bzip2      1.0    17600.0  3.416012             3.411828   
..              ...      ...        ...       ...                  ...   
562   SPEC2K17 (18)     16.0     8800.0       NaN                  NaN   
563         TPC (4)     16.0     8800.0       NaN                  NaN   
564      Hadoop (3)     16.0     8800.0       NaN                  NaN   
565  MediaBench (3)     16.0     8800.0       NaN                  NaN   
566        YCSB (6)     16.0     8800.0       NaN                  NaN   

     PRAC_WO_Mitigation  PRAC_WO_Mitigation-ClosedCap4  \
0              3.197581                       3.19726

**4-Cores Results CAP1**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

multi_cores_out_path = '../TimingOverhead/4cores_results'

df = pd.DataFrame(columns=["mitigation", "workload"])
mitigation_list = ["Baseline-ClosedCap1", "PRAC_WO_Mitigation-ClosedCap1"]
# mitigation_list = ["Baseline-ClosedCap1"]
for mitigation in mitigation_list:
    result_path = multi_cores_out_path + "/" + mitigation +"/stats/"
    result_list = [x[:-4] for x in os.listdir(result_path) if x.endswith(".txt")]
    for result_filename in result_list:
        result_file = open(result_path + result_filename + ".txt", "r")
        Channel = int(result_filename.split("_")[0])

        workload = "_".join(result_filename.split("_")[2:])

        w0=''
        w1=''
        w2=''
        w3=''
        ipc_0 = 0
        ipc_1 = 0
        ipc_2 = 0
        ipc_3 = 0
        cycle_0 = 0
        cycle_1 = 0
        cycle_2 = 0
        cycle_3 = 0
        num_inst_0=0
        num_inst_1=0
        num_inst_2=0
        num_inst_3=0
        num_rd_reqs=0
        num_wr_reqs=0
        wr_reqs_ratio = 0.0
        # num_tREFI_period=0
        # num_tREFW_period=0
        for line in result_file.readlines():
            if ("name_trace_0:" in line):
                w0 = str(line.split("/")[-1]).strip()
            if ("name_trace_1:" in line):
                w1 = str(line.split("/")[-1]).strip()
            if ("name_trace_2:" in line):
                w2 = str(line.split("/")[-1]).strip()
            if ("name_trace_3:" in line):
                w3 = str(line.split("/")[-1]).strip()
            if (" cycles_recorded_core_0:" in line):
                cycle_0 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_1:" in line):
                cycle_1 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_2:" in line):
                cycle_2 = int(line.split(" ")[-1])
            if (" cycles_recorded_core_3:" in line):
                cycle_3 = int(line.split(" ")[-1])
            if (" insts_recorded_core_0" in line):
                num_inst_0 = int(line.split(" ")[-1])
            if (" insts_recorded_core_1" in line):
                num_inst_1 = int(line.split(" ")[-1])
            if (" insts_recorded_core_2" in line):
                num_inst_2 = int(line.split(" ")[-1])
            if (" insts_recorded_core_3" in line):
                num_inst_3 = int(line.split(" ")[-1])
            if (" total_num_read_requests" in line):
                num_rd_reqs = int(line.split(" ")[-1])
            if (" total_num_write_requests" in line):
                num_wr_reqs = int(line.split(" ")[-1])
            # if (" prac_num_recovery" in line):
            #     num_abo = int(line.split(" ")[-1])
            # if (" num_refresh_command_0" in line):
            #     num_tREFI_period = int(line.split(" ")[-1])
            # if (" num_refresh_window_0" in line):
            #     num_tREFW_period = int(line.split(" ")[-1])            
                
        if (cycle_0 == 0 and cycle_1 == 0 and cycle_2 == 0 and cycle_3 == 0):
            continue
        if (cycle_0 == 0 or cycle_1 == 0 or cycle_2 == 0 or cycle_3 == 0):
            print("Error: " + result_filename)
        ipc_0 = int(num_inst_0) / cycle_0
        ipc_1 = int(num_inst_1) / cycle_1
        ipc_2 = int(num_inst_2) / cycle_2
        ipc_3 = int(num_inst_3) / cycle_3
        
        wr_reqs_ratio = float(int(num_wr_reqs)/int(num_rd_reqs + num_wr_reqs))
        result_file.close()
        # Create a new DataFrame for the new row
        new_row = pd.DataFrame({
            'mitigation': [mitigation],
            'workload': [workload],
            'Channel': [Channel],
            'wl0': [w0],
            'wl1': [w1],
            'wl2': [w2],
            'wl3': [w3],
            'ipc0': [ipc_0],
            'ipc1': [ipc_1],
            'ipc2': [ipc_2],
            'ipc3': [ipc_3],
            'RD_REQs': [num_rd_reqs],
            'WR_REQs': [num_wr_reqs],
            'WR_ratio': [wr_reqs_ratio],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# print(df)
df[['workload', 'Channel', 'mitigation', 'RD_REQs', 'WR_REQs', 'WR_ratio']].to_csv('./stats/4core_heterogeneous_workload_characteristics.csv', index=False)
df_sc_ipc = pd.read_csv('./SC_500M_Closed_Cap1_IPC.csv')
# print(df_sc_ipc)

# First, merge df with df_sc_ipc for each workload (wl0, wl1, wl2, wl3)
df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline-ClosedCap1']], left_on=['Channel', 'wl0'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap1': 'ipc_wl0'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline-ClosedCap1']], left_on=['Channel', 'wl1'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap1': 'ipc_wl1'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline-ClosedCap1']], left_on=['Channel', 'wl2'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap1': 'ipc_wl2'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Drop 'workload' again if it exists

df = df.merge(df_sc_ipc[['Channel', 'workload_sc', 'Baseline-ClosedCap1']], left_on=['Channel', 'wl3'], right_on=['Channel', 'workload_sc'], how='left').rename(columns={'Baseline-ClosedCap1': 'ipc_wl3'})
if 'workload_sc' in df.columns:
    df = df.drop(columns=['workload_sc'])  # Final cleanup

df['normalzied_ipc0'] = df['ipc0'] / df['ipc_wl0']
df['normalzied_ipc1'] = df['ipc1'] / df['ipc_wl1']
df['normalzied_ipc2'] = df['ipc2'] / df['ipc_wl2']
df['normalzied_ipc3'] = df['ipc3'] / df['ipc_wl3']

df['WS'] = df[['normalzied_ipc0', 'normalzied_ipc1', 'normalzied_ipc2', 'normalzied_ipc3']].sum(axis=1)

df_closed_cap1_ws = df[['mitigation', 'workload', 'Channel', 'WS']]

df_closed_cap1_ws_pivot = df_closed_cap1_ws.pivot(index=['workload', 'Channel'], columns=['mitigation'], values='WS').reset_index()
for mitigation in set(mitigation_list) - set(['Baseline-ClosedCap1']):
     df_closed_cap1_ws_pivot[mitigation] = df_closed_cap1_ws_pivot[mitigation] / df_closed_cap1_ws_pivot['Baseline-ClosedCap1']
df_closed_cap1_ws_pivot.drop(columns=['Baseline-ClosedCap1'], inplace=True)
##### Calculate the Geomean for each workload type
# Define benchmark suites and their corresponding workloads ranges
benchmark_types = {
    'HHHH (15)': list(range(0, 15)),   # From Mix0-Mix14
    'MMMM (15)': list(range(15, 30)),  # From Mix15-Mix29
    'LLLL (15)': list(range(30, 45)),  # From Mix30-Mix44
    'HHMM (15)': list(range(45, 60)),  # From Mix45-Mix59
    'HHLL (15)': list(range(60, 75)),  # From Mix60-Mix74
    'MMLL (15)': list(range(75, 90)),  # From Mix75-Mix89
}

# DataFrame Example (you already have your df)
# Assuming your column of interest is 'PRAC_WO_Mitigation-ClosedCap1'

# Add a new column to assign each Mix to the appropriate suite
def assign_benchmark_types(mix_index):
    for suite, mix_range in benchmark_types.items():
        if mix_index in mix_range:
            return suite
    return None

# Assuming 'workload' has values like 'MIX0', 'MIX1', etc., you can extract the index
df_closed_cap1_ws_pivot['Mix_index'] = df_closed_cap1_ws_pivot['workload'].str.extract(r'(\d+)').astype(int)  # Extract Mix number
df_closed_cap1_ws_pivot['Benchmark_Types'] = df_closed_cap1_ws_pivot['Mix_index'].apply(assign_benchmark_types)  # Assign benchmark suite

# Function to calculate the geometric mean
def geom_mean(series):
    return np.exp(np.log(series).mean())

# Function to calculate and add geometric means as new rows
def add_geomean_rows(df):
    geomean_rows = []  # List to collect new rows

    for Channel in df['Channel'].unique():
        for suite_name, mix_indices in benchmark_types.items():
            # Create a list of corresponding workload names (e.g., MIX0, MIX1)
            workloads = [f'MIX{i}' for i in mix_indices]
            suite_df = df[df['workload'].isin(workloads) & (df['Channel'] == Channel)]
            if not suite_df.empty:
                geomeans = {}
                
                # Dynamically calculate geometric means for each mitigation
                for mitigation in mitigation_list:
                    geomeans[mitigation] = calculate_geometric_mean(suite_df[mitigation])
                
                # Create a new row
                geomean_row = {'Channel': Channel, 'workload': suite_name, **geomeans}
                geomean_rows.append(geomean_row)  # Append to the list

    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

def add_all_workloads_geomean_rows(df):
    geomean_rows = []  # List to collect new rows
    
    for Channel in df['Channel'].unique():
        Channel_df = df[df['Channel'] == Channel]
        geomean_values = {}
        
        # Calculate geometric means for each mitigation in the list
        for mitigation in mitigation_list:
            if mitigation in Channel_df.columns:  # Ensure the column exists
                geomean_values[mitigation] = calculate_geometric_mean(Channel_df[mitigation])

        # Create a new row for the combined results
        geomean_row = {'Channel': Channel, 'workload': 'All (90)', **geomean_values}
        geomean_rows.append(geomean_row)  # Append to the list
    
    # Convert list of rows to DataFrame
    geomean_df = pd.DataFrame(geomean_rows)
    
    return pd.concat([df, geomean_df], ignore_index=True)

mitigation_list = ["PRAC_WO_Mitigation-ClosedCap1"]


geomean_df = add_geomean_rows(df_closed_cap1_ws_pivot)
geomean_df = add_all_workloads_geomean_rows(geomean_df)
print(geomean_df)
geomean_df.to_csv('./4cores_500M_Closed_Cap1_Result.csv', index=False)


NameError: name 'calculate_geometric_mean' is not defined